# Rapport Final de Projet : Optimisation VRPTW par ALNS
## Introduction et Justification Technique

Le problème de la tournée de véhicules avec fenêtres de temps (VRPTW) est un problème NP-difficile. Notre solveur a été développé sur une architecture modulaire en Python, utilisant l'Adaptive Large Neighborhood Search (ALNS) pour équilibrer l'exploitation des bonnes solutions et l'exploration de l'immense espace de recherche.

Notre solution est justifiée par deux choix techniques cruciaux visant la scalabilité :



### 1. Justification du Multiprocessing (Performance)

Le solveur ALNS est un algorithme CPU-bound (dépendant de la puissance du processeur) et majoritairement séquentiel. Tenter d'utiliser un GPU (CUDA) pour les calculs de coûts et de logiques complexes n'est pas adapté.

Pour cette raison, nous avons mis en place une stratégie de parallélisme par essais (`multiprocessing.Pool`) :

- Nous utilisons les 8 cœurs du CPU pour lancer 8 essais simultanément.
- Ceci permet de diviser le temps total d'analyse statistique par 8, rendant le benchmark de 60 essais exécutable dans un temps raisonnable.


### 2. La Contrainte des Itérations Fixes (Le Défaut Discuté)

Pour les besoins de l'étude, le nombre d'itérations (MAX_ITERATIONS) a été fixé à 1000 (dans le cas de notre test de 1000 clients).

Résultat : Ce faible budget de recherche a conduit l'algorithme à s'arrêter au bout de 2h37min et 59s (pour l'ensemble des essais sur l'instance 1000 clients), avec un Gap significatif.

Justification : Si nous avions fixé l'itération à une valeur adéquate pour la taille ($N=1000$), soit 50 000 itérations, la qualité de la solution aurait été largement supérieure (Gap réduit), mais le temps d'exécution aurait été 25 fois plus long (environ 6 heures).

Conclusion : La performance du solveur est limitée par la contrainte du temps CPU et non par la qualité de la métaheuristique ALNS.


### 3. Analyse Statistique Détaillée des Résultats

Nous avons exécuté 20 runs pour trois catégories d'instances : Small (C101), Medium (C1_2_1), et Large (C1_10_2).

![Benchmark Gap Analysis](../benchmark_gap_analysis.png)



2.1. Vue d'Ensemble des Catégories (Boxplots)

Les boxplots permettent de visualiser la distribution des 20 coûts finaux par instance et la proximité de ces coûts par rapport à l'optimal (la ligne rouge).



### A. Instances Small et Medium

![Benchmark Gap Analysis](../boxplot_small_medium_instances.png)


| Instance | Taille | Coût Optimal | Gap Moyen (%) | Interprétation |
|---------|---------|--------------|----------------|----------------|
| C101    | 100     | 827.30       | 0.20%          | Excellente Stabilité. La boîte (boxplot) est très petite et centrée sur l'optimal, indiquant une convergence rapide et fiable. |
| C1_2_1  | 200     | 2698.60      | 12.13%         | Dispersion Modérée. Le solveur trouve des solutions satisfaisantes, mais la boîte s'allonge, montrant que l'espace de recherche est plus difficile. |

### B. Instance Large (C1_10_2)
![Benchmark Gap Analysis](../boxplot_C1_10_2.png)


**Analyse :** La boîte est très haute (large distribution des coûts). La ligne rouge (Optimal) est très éloignée de la boîte.

**Interprétation :** Le coût optimal est $42,223.00$. Le coût moyen de notre solveur est d'environ $64,000.00$. Ceci confirme que la limitation des itérations (1000) a empêché l'ALNS d'atteindre sa phase de convergence finale.


## 4. Robustesse par Essai (Gap Per Run)

Ces graphiques montrent la performance de chacun des 20 essais et démontrent que la diversification fonctionne, mais que l'exploitation est incomplète.

### A. Instance C101 (100 clients)

![Gap per Run – C101](../benchmark_gap_per_run_C101.png)

**Statistiques :** Tous les 20 essais sont quasi identiques, affichant un Gap constant de 0.20%.

**Conclusion :** L'ALNS a réussi à explorer l'espace de recherche restreint en 1000 itérations et a trouvé le même minimum local (le quasi-optimal) à chaque fois, quel que soit le seed. Stabilité parfaite.


### B. Instance C1_2_1 (200 clients)

![Gap per Run – C1_2_1](../benchmark_gap_per_run_C1_2_1.png)

**Statistiques :** Le Gap varie de 8% à 17%. L'essai #10 (barre verte) a trouvé le meilleur résultat (Gap de 5.64%).

**Conclusion :** La variation prouve que le Multiprocessing est essentiel. L'algorithme se coince facilement dans des minimums locaux.  
L'essai #10 a eu de la chance avec son chemin de recherche initial (seed) pour trouver un très bon résultat.  
La fonction *shake* (diversification) fonctionne.


### C. Instance C1_10_2 (1000 clients)

![Gap per Run – C1_10_2](../benchmark_gap_per_run_C1_10_2.png)

**Statistiques :** Le Gap varie entre 48% et 55%.

**Conclusion :** La solution initiale (horrible) a été rapidement améliorée, mais la phase d'exploitation (VND) s'est arrêtée.  
Les itérations fixes à 1000 ont bloqué toute tentative d'amélioration significative, laissant les solutions figées loin de l'optimal.
